In [1]:
import numpy as np #linear algebra
import pandas as pd #data processing 
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

# Importing into dataframe
df_Bom_MovieGross = pd.read_csv("./data/DownLoadedData_FlatIron/bom.movie_gross.csv.gz", delimiter=',', encoding='ANSI')
df_TN_Movie_Budgets = pd.read_csv("./data/DownLoadedData_FlatIron/tn.movie_budgets.csv.gz", delimiter=',')
df_TN_Movie_Budgets = pd.read_csv("./data/DownLoadedData_FlatIron/tn.movie_budgets.csv.gz", delimiter=',')
df_Scrp_Financials = pd.read_excel("./data/ImdbScrapingData/df_Financials_SummaryAndDetail_Imdb_Dom.xlsx")
df_IMDB_Akas_english = pd.read_excel("./data/DownLoadedData_Imdb/df_IMDB_Akas_english.xlsx")
df_Studio_x_ref_For_import = pd.read_excel("./data/ImdbScrapingData/df_Studio_x_ref_For_import.xlsx")
# df_Studio_cross_Ref_FI = pd.read_excel("./data/DownLoadedData_FlatIron/df_Studio_Cross_ref_FI.xlsx")


In [2]:
#PREPARE THE FINACIAL DATA:  WE USED SOURCES,2 FROM FLATIRON DOWNLOAD, 1 FROM SCRAPING IMDB Site :https://www.boxofficemojo.com/
#****************************************************************************************
#prepraring to work with Movie Gross Df, cleaning up data and create right datatypes
df_Bom_MovieGross.name = "df_Bom_MovieGross"
df_Bom_MovieGross['foreign_gross'] = pd.to_numeric(df_Bom_MovieGross['foreign_gross'],errors='coerce')
df_Bom_MovieGross['year_str_BOM'] = df_Bom_MovieGross['year'].astype(str)
df_Bom_MovieGross['year'] = df_Bom_MovieGross['year'].astype(int)
df_Bom_MovieGross_values = {'domestic_gross':0, 'foreign_gross': 0}
df_Bom_MovieGross.fillna(value=df_Bom_MovieGross_values, inplace=True)
df_Bom_MovieGross["wwg_calc_BOM"] = 0 
df_Bom_MovieGross['title'] = df_Bom_MovieGross['title'].str.title() #******KEY STEP
df_Bom_MovieGross["titleyear"] = df_Bom_MovieGross['title'] + df_Bom_MovieGross['year_str_BOM']
df_Bom_MovieGross["wwg_calc_BOM"] = (df_Bom_MovieGross['domestic_gross'] + df_Bom_MovieGross['foreign_gross'])
#Renaming to aid in consolidating between the three sources

# #getting the cross ref data for studio so I can have = to scrapped data
# df_Bom_MovieGross_withStudio = df_Bom_MovieGross.merge(df_Studio_x_ref_For_import, 
#                                                left_on='studio', right_on="studio_BOM1",how='left', 
#                                             indicator='mergingstudio', indicator="checking" suffixes=('_fl', '_IM'))

# df_Bom_MovieGross_withStudio.drop(['mergingstudio','studio_BOM'], axis=1, inplace=True)

df_Bom_MovieGross.rename(columns={"foreign_gross": "fg_BOM", "domestic_gross": "dg_BOM","year": "year_BOM",
                                             "title":"title_BOM","studio":"studio_BOM" },inplace=True)
#-------------------------------------------------------------

#prepraring to work with Movie budgets, cleaning up data and create right datatypes
df_TN_Movie_Budgets.name = "df_TN_Movie_Budgets"
df_TN_Movie_Budgets[df_TN_Movie_Budgets.columns[1:]] = df_TN_Movie_Budgets[df_TN_Movie_Budgets.columns[1:]].replace('[\$,]', '', regex=True)
df_TN_Movie_Budgets['production_budget'] = pd.to_numeric(df_TN_Movie_Budgets['production_budget'],errors='coerce')
df_TN_Movie_Budgets['domestic_gross'] = pd.to_numeric(df_TN_Movie_Budgets['domestic_gross'],errors='coerce')
df_TN_Movie_Budgets['worldwide_gross'] = pd.to_numeric(df_TN_Movie_Budgets['worldwide_gross'],errors='coerce')

df_TN_Movie_Budgets_values = {'domestic_gross': 0, 'year':0 , 'worldwide_gross': 0}
df_TN_Movie_Budgets.fillna(value=df_TN_Movie_Budgets_values, inplace=True)
df_TN_Movie_Budgets["fg_calc"] = 0

df_TN_Movie_Budgets['movie'] = df_TN_Movie_Budgets['movie'].str.title() #******KEY STEP
df_TN_Movie_Budgets["year"] = df_TN_Movie_Budgets['release_date'].str[-4:].astype(int)
df_TN_Movie_Budgets["year_str_TN"] = df_TN_Movie_Budgets['release_date'].str[-4:]
df_TN_Movie_Budgets["titleyear"] = df_TN_Movie_Budgets['movie'] + df_TN_Movie_Budgets['year_str_TN']
df_TN_Movie_Budgets["fg_calc_TN"] = (df_TN_Movie_Budgets['worldwide_gross'] - df_TN_Movie_Budgets['domestic_gross'])
df_TN_Movie_Budgets = df_TN_Movie_Budgets.drop('id', axis = 1)
df_TN_Movie_Budgets.rename(columns={"domestic_gross": "dg_TN", "worldwide_gross": "wwg_TN","year": "year_TN", 
                                    "production_budget": "pb_TN" ,"movie":"title_TN","release_date":"rd_TN" }, inplace=True)

#needed to do this steep to collapse data given there are dups in a few movienames and titles - ?bluebeard
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
q3 = """SELECT titleyear as titleyear, max(title_TN) as title_TN, max(year_TN) as year_TN, min(year_str_TN) as year_str_TN, 
min(rd_TN) as rd_TN, Sum(pb_TN) as pb_TN,sum(dg_TN) as dg_TN, sum(wwg_TN) as wwg_TN , sum(fg_calc_TN) as fg_calc_TN
        FROM df_TN_Movie_Budgets
        GROUP BY titleyear, title_TN
        """
df_TN_Movie_Budgets = pysqldf(q3)
#-------------------------------------------------------------

# Finanally merger data from flatiron
df_fI_Financials = df_TN_Movie_Budgets.merge(df_Bom_MovieGross, 
     on='titleyear', how='outer', indicator='Combing_FL_Financials', suffixes=('_TN', '_Bom'))

# Add tconst Key to FLatIronTables then drop 
df_FI_financials_With_tconst = df_fI_Financials.merge(df_IMDB_Akas_english[['tconst','titleyear']], on='titleyear', how="inner", indicator='Adding tconst', suffixes=('_fi', '_im'))



In [3]:
df_fI_Financials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7927 entries, 0 to 7926
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   titleyear              7927 non-null   object  
 1   title_TN               5781 non-null   object  
 2   year_TN                5781 non-null   float64 
 3   year_str_TN            5781 non-null   object  
 4   rd_TN                  5781 non-null   object  
 5   pb_TN                  5781 non-null   float64 
 6   dg_TN                  5781 non-null   float64 
 7   wwg_TN                 5781 non-null   float64 
 8   fg_calc_TN             5781 non-null   float64 
 9   title_BOM              3387 non-null   object  
 10  studio_BOM             3382 non-null   object  
 11  dg_BOM                 3387 non-null   float64 
 12  fg_BOM                 3387 non-null   float64 
 13  year_BOM               3387 non-null   float64 
 14  year_str_BOM           3387 non-null   o

In [4]:
df_FI_financials_With_tconst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2609 entries, 0 to 2608
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   titleyear              2609 non-null   object  
 1   title_TN               1983 non-null   object  
 2   year_TN                1983 non-null   float64 
 3   year_str_TN            1983 non-null   object  
 4   rd_TN                  1983 non-null   object  
 5   pb_TN                  1983 non-null   float64 
 6   dg_TN                  1983 non-null   float64 
 7   wwg_TN                 1983 non-null   float64 
 8   fg_calc_TN             1983 non-null   float64 
 9   title_BOM              1647 non-null   object  
 10  studio_BOM             1646 non-null   object  
 11  dg_BOM                 1647 non-null   float64 
 12  fg_BOM                 1647 non-null   float64 
 13  year_BOM               1647 non-null   float64 
 14  year_str_BOM           1647 non-null   o

In [5]:
df_FI_financials_With_tconst.nunique()

titleyear                2594
title_TN                 1963
year_TN                    15
year_str_TN                15
rd_TN                     875
pb_TN                     278
dg_TN                    1902
wwg_TN                   1938
fg_calc_TN               1874
title_BOM                1640
studio_BOM                131
dg_BOM                   1096
fg_BOM                    870
year_BOM                    9
year_str_BOM                9
wwg_calc_BOM             1332
Combing_FL_Financials       3
tconst                   2609
Adding tconst               1
dtype: int64

In [6]:
# # df_FI_financials_With_tconst[["title_TN","title_BOM","title_FI",]].sort_values(by="title_FI", ascending=False)
# df_FI_financials_With_tconst[["year_TN","year_BOM","year_str_FI",]].sort_values(by="year_str_FI", ascending=False)
# df_FI_financials_With_tconst[["year_TN","year_BOM","year_FI",]].sort_values(by="year_FI", ascending=False)
# # df_Bom_MovieGross[["studio","studio_short","Studio_Desc"]].sort_values(by="title_FI", ascending=False)

In [7]:
df_FI_financials_With_tconst.columns

Index(['titleyear', 'title_TN', 'year_TN', 'year_str_TN', 'rd_TN', 'pb_TN',
       'dg_TN', 'wwg_TN', 'fg_calc_TN', 'title_BOM', 'studio_BOM', 'dg_BOM',
       'fg_BOM', 'year_BOM', 'year_str_BOM', 'wwg_calc_BOM',
       'Combing_FL_Financials', 'tconst', 'Adding tconst'],
      dtype='object')

In [8]:
#Use all data scraped from IMDB as the default for domestic, foreign ww sales as well as budget
df_FI_financials_With_tconst.title_TN.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_str_TN.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_TN.fillna(0, inplace=True)

df_FI_financials_With_tconst.title_BOM.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_str_BOM.fillna("0", inplace=True)
df_FI_financials_With_tconst.year_BOM.fillna(0, inplace=True)

df_FI_financials_With_tconst['title_FI'] = df_FI_financials_With_tconst['title_TN']
df_FI_financials_With_tconst['year_str_FI'] = df_FI_financials_With_tconst['year_str_TN']
df_FI_financials_With_tconst['year_FI'] = df_FI_financials_With_tconst['year_TN']

df_FI_financials_With_tconst['title_FI'] = df_FI_financials_With_tconst.title_FI.fillna("0").astype(str)
df_FI_financials_With_tconst['year_str_FI'] = df_FI_financials_With_tconst.year_str_FI.fillna("0").astype(str)
df_FI_financials_With_tconst['year_FI'] = df_FI_financials_With_tconst.year_FI.fillna(0).astype(int)

df_FI_financials_With_tconst['title_FI'] = df_FI_financials_With_tconst.apply(lambda x: x.title_BOM if x.title_FI == "0" else x.title_FI, axis=1)
df_FI_financials_With_tconst['year_str_FI'] = df_FI_financials_With_tconst.apply(lambda x: x.year_str_BOM if x.year_str_FI == "0" else x.year_str_FI, axis=1)
df_FI_financials_With_tconst['year_FI'] = df_FI_financials_With_tconst.apply(lambda x: x.year_BOM if x.year_FI == 0 else x.year_FI, axis=1)


In [9]:
df_FI_financials_With_tconst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2609 entries, 0 to 2608
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   titleyear              2609 non-null   object  
 1   title_TN               2609 non-null   object  
 2   year_TN                2609 non-null   float64 
 3   year_str_TN            2609 non-null   object  
 4   rd_TN                  1983 non-null   object  
 5   pb_TN                  1983 non-null   float64 
 6   dg_TN                  1983 non-null   float64 
 7   wwg_TN                 1983 non-null   float64 
 8   fg_calc_TN             1983 non-null   float64 
 9   title_BOM              2609 non-null   object  
 10  studio_BOM             1646 non-null   object  
 11  dg_BOM                 1647 non-null   float64 
 12  fg_BOM                 1647 non-null   float64 
 13  year_BOM               2609 non-null   float64 
 14  year_str_BOM           2609 non-null   o

In [10]:
# df_FI_financials_With_tconst.drop(["year_TN","year_BOM","year_TN","year_BOM", "year_str_TN","year_str_BOM","Combing_FL_Financials"], axis=1, inplace=True)

In [11]:
df_FI_financials_With_tconst.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2609 entries, 0 to 2608
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   titleyear              2609 non-null   object  
 1   title_TN               2609 non-null   object  
 2   year_TN                2609 non-null   float64 
 3   year_str_TN            2609 non-null   object  
 4   rd_TN                  1983 non-null   object  
 5   pb_TN                  1983 non-null   float64 
 6   dg_TN                  1983 non-null   float64 
 7   wwg_TN                 1983 non-null   float64 
 8   fg_calc_TN             1983 non-null   float64 
 9   title_BOM              2609 non-null   object  
 10  studio_BOM             1646 non-null   object  
 11  dg_BOM                 1647 non-null   float64 
 12  fg_BOM                 1647 non-null   float64 
 13  year_BOM               2609 non-null   float64 
 14  year_str_BOM           2609 non-null   o

In [12]:
df_Scrp_Financials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4920 entries, 0 to 4919
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              4920 non-null   int64  
 1   tconst             4920 non-null   object 
 2   dg_IMDB            4920 non-null   int64  
 3   fg_IMDB            4920 non-null   int64  
 4   ww_IMDB            4920 non-null   int64  
 5   Domestic Opening   4592 non-null   float64
 6   pb_IMDB            1461 non-null   float64
 7   rd_IMDB            4920 non-null   object 
 8   MPAA               3315 non-null   object 
 9   Running Time_IMDB  4915 non-null   object 
 10  studio_IMDB        4754 non-null   object 
 11  Genres_IMDB        4920 non-null   object 
 12  title_IMDB         4920 non-null   object 
 13  originalTitle      4920 non-null   object 
 14  isAdult            4920 non-null   int64  
 15  year               4920 non-null   int64  
 16  runtimeMinutes     4920 

In [13]:
#Merg flatiron financials with scraping finacials IMDB Site :https://www.boxofficemojo.com/
#****************************************************************************************
#prepraring to work with Movie Gross Df, cleaning up data and create right datatypes
df_MasterFinancials = df_Scrp_Financials.merge(df_FI_financials_With_tconst, 
                                               on='tconst', how='outer', suffixes=('_IM', '_fl'))
fieldsToConvert = {'dg_IMDB': 0, 'fg_IMDB':0 , 'ww_IMDB': 0, 'Domestic Opening': 0, 'pb_IMDB':0 , 'pb_TN': 0,
                       'dg_TN': 0, 'wwg_TN':0 , 'fg_calc_TN': 0,'dg_BOM': 0, 'fg_BOM':0 , 'wwg_calc_BOM': 0,
                      'wg_calc_BOM': 0}
df_MasterFinancials.fillna(value=fieldsToConvert, inplace=True)

In [14]:
df_MasterFinancials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5540 entries, 0 to 5539
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   index                  4920 non-null   float64 
 1   tconst                 5540 non-null   object  
 2   dg_IMDB                5540 non-null   float64 
 3   fg_IMDB                5540 non-null   float64 
 4   ww_IMDB                5540 non-null   float64 
 5   Domestic Opening       5540 non-null   float64 
 6   pb_IMDB                5540 non-null   float64 
 7   rd_IMDB                4920 non-null   object  
 8   MPAA                   3315 non-null   object  
 9   Running Time_IMDB      4915 non-null   object  
 10  studio_IMDB            4754 non-null   object  
 11  Genres_IMDB            4920 non-null   object  
 12  title_IMDB             4920 non-null   object  
 13  originalTitle          4920 non-null   object  
 14  isAdult                4920 non-null   f

In [15]:
df_MasterFinancials.columns

Index(['index', 'tconst', 'dg_IMDB', 'fg_IMDB', 'ww_IMDB', 'Domestic Opening',
       'pb_IMDB', 'rd_IMDB', 'MPAA', 'Running Time_IMDB', 'studio_IMDB',
       'Genres_IMDB', 'title_IMDB', 'originalTitle', 'isAdult', 'year',
       'runtimeMinutes', 'genres', 'titleyear_IM', 'titleyear_fl', 'title_TN',
       'year_TN', 'year_str_TN', 'rd_TN', 'pb_TN', 'dg_TN', 'wwg_TN',
       'fg_calc_TN', 'title_BOM', 'studio_BOM', 'dg_BOM', 'fg_BOM', 'year_BOM',
       'year_str_BOM', 'wwg_calc_BOM', 'Combing_FL_Financials',
       'Adding tconst', 'title_FI', 'year_str_FI', 'year_FI'],
      dtype='object')

In [16]:
#Clean up dataframe - consolidate, remove unwanted columns, all remove metadata from financials
df_MasterFinancials = df_MasterFinancials.reindex(sorted(df_MasterFinancials.columns), axis=1)
col_names = ['tconst','Adding tconst', 'Combing_FL_Financials',
       'Genres_IMDB', 'MPAA', 'Running Time_IMDB', 'Unnamed: 0', 'Domestic Opening','dg_BOM',
       'dg_IMDB', 'dg_TN', 'fg_BOM', 'fg_IMDB', 'fg_calc_TN', 'genres',
       'isAdult', 'mergingfinancials', 'originalTitle', 'pb_IMDB', 'pb_TN',
       'rd_IMDB', 'rd_TN', 'runtimeMinutes', 'studio_BOM', 'studio_IMDB',
        'title_BOM', 'title_IMDB', 'title_TN', 'titleyear_IM',
       'titleyear_fl', 'wwg_calc_BOM', 'ww_IMDB', 'wwg_TN',
       'year', 'year_BOM', 'year_TN']
df_MasterFinancials.reindex(columns=col_names)


# df_MasterFinancials.drop(["Adding tconst", 'Unnamed: 0', 'isAdult', 'mergingfinancials'], axis=1,inplace = True )

df_MasterFinancials_Only = df_MasterFinancials[['tconst','Domestic Opening','dg_BOM', 'dg_TN', 'dg_IMDB',
                                                'fg_BOM', 'fg_calc_TN','fg_IMDB', 'pb_IMDB', 'pb_TN',
                                                'title_BOM', 'title_IMDB', 'title_TN', 'titleyear_IM','titleyear_fl',
                                                'wwg_calc_BOM', 'wwg_TN','ww_IMDB','year']]


    


# df_FI_financials_With_tconst.drop(["year_TN","year_BOM","year_TN","year_BOM", "year_str_TN","year_str_BOM","Combing_FL_Financials"], axis=1, inplace=True)


In [17]:
df_MasterFinancials_MetaData.sort_values(by="year")

NameError: name 'df_MasterFinancials_MetaData' is not defined

In [ ]:
#Use all data scraped from IMDB as the default for domestic, foreign ww sales as well as budget
df_MasterFinancials_Only['WW_Gross'] = df_MasterFinancials_Only['ww_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['Dom_Gross'] = df_MasterFinancials_Only['dg_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['Frgn_Gross'] = df_MasterFinancials_Only['fg_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['P_Cost'] = df_MasterFinancials_Only['pb_IMDB'].apply(lambda x: x if x>0 else 0)

#fill in the blanks from max valus from the flatIron finance tables
df_MasterFinancials_Only['dg_Fl_Max'] = df_MasterFinancials_Only[["dg_BOM", "dg_TN"]].max(axis=1)
df_MasterFinancials_Only['fg_Fl_Max'] = df_MasterFinancials_Only[["fg_BOM", "fg_calc_TN"]].max(axis=1)
df_MasterFinancials_Only['ww_Fl_Max'] = df_MasterFinancials_Only['dg_Fl_Max']+df_MasterFinancials_Only['fg_Fl_Max']

df_MasterFinancials_Only['WW_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.ww_Fl_Max if x.WW_Gross==0 else x.WW_Gross, axis=1)
df_MasterFinancials_Only['Dom_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.dg_Fl_Max if x.Dom_Gross==0 else x.Dom_Gross, axis=1)
df_MasterFinancials_Only['Frgn_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.fg_Fl_Max if x.Frgn_Gross==0 else x.Frgn_Gross, axis=1)
df_MasterFinancials_Only['P_Cost'] = df_MasterFinancials_Only.apply(lambda x: x.pb_TN if x.P_Cost==0 else x.P_Cost, axis=1)
df_MasterFinancials_Only['P_Cost']=df_MasterFinancials_Only['P_Cost'].replace(0,df_MasterFinancials_Only.P_Cost.mean())

df_MasterFinancials_Only['titleyear'] = df_MasterFinancials_Only["titleyear_IM"]
df_MasterFinancials_Only.titleyear.fillna(df_MasterFinancials_Only["titleyear_fl"],inplace=True)

df_MasterFinancials_MetaData['titleyear'] = df_MasterFinancials_Only["titleyear_IM"]
df_MasterFinancials_MetaData.titleyear.fillna(df_MasterFinancials_Only["titleyear_fl"],inplace=True)



In [ ]:
df_MasterFinancials_Only = df_MasterFinancials_Only[['tconst','P_Cost','Dom_Gross','Frgn_Gross', 'WW_Gross']]

In [ ]:
df_MasterFinancials_Only.info()

In [ ]:
df_MasterFinancials_Only.to_excel("df_MasterFinancials_Only.xlsx")

---------------------------------------------------------------------------------------------------------
The below data is used to get the metadata that is in the finance tables like run-time, release data, Studio etc.

In [52]:
df_MasterFinancials_MetaData = df_MasterFinancials[['tconst', 'Domestic Opening',
                                                    'Genres_IMDB','genres', 'MPAA', 'Running Time_IMDB','runtimeMinutes', 
                                                    'studio_IMDB', 'originalTitle', 'rd_IMDB','rd_TN',
                                                    'studio_BOM', 'studio_IMDB','title_BOM', 'title_IMDB', 'title_TN',
                                                    'titleyear_IM','titleyear_fl', 'ww_IMDB',
                                                    'year','year_BOM', 'year_TN']]

df_MasterFinancials_MetaData["Genres_S"] = df_MasterFinancials_MetaData.Genres_IMDB.replace('\n','', regex=True)
df_MasterFinancials_MetaData["Genres_S"].fillna("", inplace=True)
df_MasterFinancials_MetaData["Genres_S"] = df_MasterFinancials_MetaData["Genres_S"].str.replace('           ',',', regex=True)
# df_MasterFinancials_MetaData["Genres_S"] = df_MasterFinancials_MetaData["Genres_S"].apply(lambda x: x.strip())

<ipython-input-52-2bd22b8bf0ed>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MasterFinancials_MetaData["Genres_S"] = df_MasterFinancials_MetaData.Genres_IMDB.replace('\n','', regex=True)
C:\Users\rgpih\anaconda3a\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-52-2bd22b8bf0ed>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [45]:
df_MasterFinancials_MetaData["year"].sort_values().fillna(0, inplace=True)
# df_MasterFinancials_MetaData["year"].sort_values()

In [33]:

# df_MasterFinancials_MetaData.year_BOM.fillna(0, inplace = True)
# df_MasterFinancials_MetaData.year_TN.fillna(0, inplace = True)

# df_MasterFinancials_MetaData['year'] = df_MasterFinancials_MetaData.year.fillna(0)
# df_MasterFinancials_MetaData['year_BOM'] = df_MasterFinancials_MetaData.year_BOM.fillna(0)
# df_MasterFinancials_MetaData['year_TN'] = df_MasterFinancials_MetaData.year_TN.fillna(0)

# df_MasterFinancials_MetaData['year_fin'] = df_MasterFinancials_MetaData['year']|
# df_MasterFinancials_MetaData.year_fin.fillna(0, inplace=True)
# df_MasterFinancials_MetaData['year_fin'] = df_MasterFinancials_MetaData.apply(lambda x:[x.year_TN, x.year_BOM].max(axis=1) if x.year_fin ==0 else x.year_fin, axis=1)

In [54]:
# df_MasterFinancials_MetaData[["title_TN","title_BOM","title_IMDB","title_fin"]].sort_values(by="title_fin", ascending=False)
# df_MasterFinancials_MetaData[["genres_fin","genres",]].sort_values(by="genres_fin", ascending=False)
# df_MasterFinancials_MetaData[["year_BOM","year_TN","year"]].sort_values(by="year", ascending=False)
df_MasterFinancials_MetaData["year"] = df_MasterFinancials_MetaData["year"].fillna(0)
df_MasterFinancials_MetaData[["year"]].sort_values(by="year")

,year
5539,0.0
5128,0.0
5127,0.0
5126,0.0
5125,0.0
5124,0.0
5123,0.0
5122,0.0
5121,0.0
5120,0.0


In [27]:
#Use all data scraped from IMDB as the default for domestic, foreign ww sales as well as budget
df_MasterFinancials_MetaData['year'] = df_MasterFinancials_MetaData['year'].fillna(0,inplace=True)

df_MasterFinancials_MetaData['titleyear_fin'] = df_MasterFinancials_MetaData['titleyear_IM']
df_MasterFinancials_MetaData['genres_fin'] = df_MasterFinancials_MetaData['Genres_S']
df_MasterFinancials_MetaData['MPAA_fin'] = df_MasterFinancials_MetaData['MPAA']
df_MasterFinancials_MetaData['RunningTime_fin'] = df_MasterFinancials_MetaData['Running Time_IMDB']
df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData['title_IMDB']

# df_MasterFinancials_MetaData.fillna("0", inplace = True)

#df_MasterFinancials_MetaData['studio_fin'] = df_MasterFinancials_MetaData['Studio_Desc_IM']

#fill in the blanks fromvalus from the flatIron data where missing
df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData['title_fin'].fillna("0")
df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_BOM if x.title_fin == "0" else x.title_fin, axis=1)
df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData['title_fin'].fillna("0")
df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_TN if x.title_fin == "0" else x.title_fin, axis=1)

df_MasterFinancials_MetaData['genres_fin'] = df_MasterFinancials_MetaData['genres_fin'].fillna("0")
df_MasterFinancials_MetaData['genres_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.genres if x.genres_fin == "0" else x.genres_fin, axis=1)


# df_MasterFinancials_MetaData.year_BOM = df_MasterFinancials_MetaData.year_BOM.fillna(0, inplace=True)
# df_MasterFinancials_MetaData.year_TN = df_MasterFinancials_MetaData.year_TN.fillna(0, inplace=True)
# df_MasterFinancials_MetaData.year_fin = df_MasterFinancials_MetaData.year_fin.fillna(0, inplace=True)
#---------------------------


# df_MasterFinancials_MetaData.year_BOM.fillna(0, inplace=True)
# df_MasterFinancials_MetaData.year_TN.fillna(0, inplace=True)
# df_MasterFinancials_MetaData.year_fin.fillna(0, inplace=True)
# df_MasterFinancials_MetaData['year_fin'] = df_MasterFinancials_MetaData['year']
# df_MasterFinancials_MetaData['year_fin'] = df_MasterFinancials_MetaData.apply(lambda x:[x.year_TN, x.year_BOM].max(axis=1) if x.year_fin ==0 else x.year_fin, axis=1)







#df_MasterFinancials_MetaData['year_fin'] = df_MasterFinancials_MetaData.apply(lambda x: 15 if x.year_fin ==0 else x.year_fin, axis=1)

# df_MasterFinancials_MetaData['year_BOM'] = df_MasterFinancials_MetaData['year_BOM']
# df_MasterFinancials_MetaData['year_TN'] = df_MasterFinancials_MetaData.to_numeric(df_MasterFinancials_MetaData['year_TN'])


# df_MasterFinancials_MetaData.year_fin = pd.to_numeric(df_MasterFinancials_MetaData.year, errors='coerce').fillna(0).astype(np.int64)
# df_MasterFinancials_MetaData['year_fin'] = df_MasterFinancials_MetaData['year']
# df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData['title_IMDB']

# df_MasterFinancials_MetaData['title_fin'].fillna("0", inplace=True) = df_MasterFinancials_MetaData['title_fin'].fillna("0", inplace=True)

# 
# df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_TN if not x.title_fin else x.title_fin, axis=1)
# df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_TN if not x.title_fin else x.title_fin, axis=1)

# df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData.apply(lambda x:[[x.title_TN, x.title_BOM]].max(axis=1) if not x.title_fin else x.title_fin, axis=1)
# df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_TN if pd.isnull(x.title_fin) else x.title_fin, axis=1)
# df_MasterFinancials_MetaData['titleyear_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_TN if  pd.isnull(x.titleyear_fin) else x.titleyear_fin, axis=1)
# df_MasterFinancials_MetaData['titleyear_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_BOM if  pd.isnull(x.titleyear_fin) else x.titleyear_fin, axis=1)


# df_MasterFinancials_MetaData['titleyear_fin'].fillna("", inplace=True)
# df_MasterFinancials_MetaData['titleyear_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_TN if x.titleyear_fin == "" else x.titleyear_fin, axis=1)
# df_MasterFinancials_MetaData['titleyear_fin'] = df_MasterFinancials_MetaData.apply(lambda x: x.title_BOM if x.titleyear_fin == "" else x.titleyear_fin, axis=1)




# title_TN
# df_MasterFinancials_Only['dg_Fl_Max'] = df_MasterFinancials_Only[["title_TN", "title_BOM"]].max(axis=1)
# df_MasterFinancials_Only['titleyear_fin'] = df_MasterFinancials_Only['titleyear_IM']
# df_MasterFinancials_Only['P_Cost'] = df_MasterFinancials_Only['pb_IMDB'].apply(lambda x: x if x>0 else 0)
# df_MasterFinancials_Only['dg_Fl_Max'] = df_MasterFinancials_Only[["dg_BOM", "dg_TN"]].max(axis=1)
# df_MasterFinancials_Only['fg_Fl_Max'] = df_MasterFinancials_Only[["fg_BOM", "fg_calc_TN"]].max(axis=1)
# df_MasterFinancials_Only['ww_Fl_Max'] = df_MasterFinancials_Only['dg_Fl_Max']+df_MasterFinancials_Only['fg_Fl_Max']

# df_MasterFinancials_Only['WW_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.ww_Fl_Max if x.WW_Gross==0 else x.WW_Gross, axis=1)




C:\Users\rgpih\anaconda3a\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-27-55950df8e637>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MasterFinancials_MetaData['year'] = df_MasterFinancials_MetaData['year'].fillna(0,inplace=True)
<ipython-input-27-55950df8e637>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [ ]:
df_MasterFinancials_MetaData
#getting the cross ref data for studio so I can have = Flatiron data
# df_MasterFinancials = df_MasterFinancials.merge(df_Studio_x_ref_For_import, 
                                               left_on='studio_IMDB_IM', right_on="studio_IMDB", how='left', 
                                            indicator='mergingstudio', suffixes=('_fl', '_IM'))

# df_MasterFinancials.drop(['studio_IMDB_IM','mergingstudio'], axis=1, inplace=True)



In [ ]:
df_MasterFinancials_MetaData['title_fin'].fillna("0", inplace=True)
# df_MasterFinancials_MetaData['title_fin'] = df_MasterFinancials_MetaData.apply(lambda x: "x.title_TN" if x.title_fin == "0" else x.title_fin, axis=1)

In [ ]:
df_MasterFinancials_MetaData[["title_fin","tconst"]].sort_values(by="title_fin",ascending=False)

In [ ]:
DfBom = df_MasterFinancials_MetaData.groupby(["studio_BOM"])["studio_BOM"].count().sort_values(ascending=False)
DfBom

DfIMDB =df_MasterFinancials_MetaData.groupby(["studio_IMDB"])["studio_IMDB"].count().sort_values(ascending=False)
DfIMDB

In [ ]:
# DfIMDB.to_excel("df_DfIMDB.xlsx")
# df_MasterFinancials_MetaData.to_excel("df_MasterFinancials_MetaData.xlsx")

In [ ]:
df_MasterFinancials_Only['Running Time_IMDB_fin'] = df_MasterFinancials_Only['dg_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['releasedate_fin'] = df_MasterFinancials_Only['fg_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['studio_fin'] = df_MasterFinancials_Only['pb_IMDB'].apply(lambda x: x if x>0 else 0)
df_MasterFinancials_Only['year_fin'] = df_MasterFinancials_Only['pb_IMDB'].apply(lambda x: x if x>0 else 0)

#fill in the blanks from max valus from the flatIron finance tables
df_MasterFinancials_Only['dg_Fl_Max'] = df_MasterFinancials_Only[["dg_BOM", "dg_TN"]].max(axis=1)
df_MasterFinancials_Only['fg_Fl_Max'] = df_MasterFinancials_Only[["fg_BOM", "fg_calc_TN"]].max(axis=1)
df_MasterFinancials_Only['ww_Fl_Max'] = df_MasterFinancials_Only['dg_Fl_Max']+df_MasterFinancials_Only['fg_Fl_Max']

df_MasterFinancials_Only['WW_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.ww_Fl_Max if x.WW_Gross==0 else x.WW_Gross, axis=1)
df_MasterFinancials_Only['Dom_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.dg_Fl_Max if x.Dom_Gross==0 else x.Dom_Gross, axis=1)
df_MasterFinancials_Only['Frgn_Gross'] = df_MasterFinancials_Only.apply(lambda x: x.fg_Fl_Max if x.Frgn_Gross==0 else x.Frgn_Gross, axis=1)
df_MasterFinancials_Only['P_Cost'] = df_MasterFinancials_Only.apply(lambda x: x.pb_TN if x.P_Cost==0 else x.P_Cost, axis=1)
df_MasterFinancials_Only['P_Cost']=df_MasterFinancials_Only['P_Cost'].replace(0,df_MasterFinancials_Only.P_Cost.mean())

df_MasterFinancials_Only['titleyear'] = df_MasterFinancials_Only["titleyear_IM"]
df_MasterFinancials_Only.titleyear.fillna(df_MasterFinancials_Only["titleyear_fl"],inplace=True)

df_MasterFinancials_MetaData['titleyear'] = df_MasterFinancials_Only["titleyear_IM"]
df_MasterFinancials_MetaData.titleyear.fillna(df_MasterFinancials_Only["titleyear_fl"],inplace=True)

In [ ]:
DfBom = df_Bom_MovieGross[["studio","studio_short","Studio_Desc"]].sort_values(by="studio", ascending=False)
DfBom
DfBom1 = df_Bom_MovieGross.groupby(["studio"])["studio"].count().sort_values(ascending=False)
DfBom1
# lst_genres = df_TitleAndRatingsAndGross.groupby(["genres"])["genres"].count().sort_values(ascending=False)